# Import Party!

In [1]:
!pip install pydicom
!pip install pytorch_lightning
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch ,torchvision,glob,os,time,pydicom,pdb
import pytorch_lightning as pl
import pydicom 
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
import pdb

     |████████████████████████████████| 2.0 MB 4.8 MB/s 
     |████████████████████████████████| 584 kB 4.9 MB/s 
     |████████████████████████████████| 136 kB 54.1 MB/s 
     |████████████████████████████████| 409 kB 48.2 MB/s 
     |████████████████████████████████| 596 kB 46.4 MB/s 
     |████████████████████████████████| 1.1 MB 47.2 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
     |████████████████████████████████| 144 kB 48.9 MB/s 
     |████████████████████████████████| 271 kB 52.5 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
def dcmtag2table(folder, list_of_tags):
    """
    # Create a Pandas DataFrame with the <list_of_tags> DICOM tags
    # from the DICOM files in <folder>
    # Parameters:
    #    folder (str): folder to be recursively walked looking for DICOM files.
    #    list_of_tags (list of strings): list of DICOM tags with no whitespaces.
    # Returns:
    #    df (DataFrame): table of DICOM tags from the files in folder.
    """
    list_of_tags = list_of_tags.copy()
    items = []
    table = []
    filelist = []
    print("Listing all files...")
    start = time.time()
    for root, dirs, files in os.walk(folder, topdown=False):
        for name in files:
            filelist.append(os.path.join(root, name))
    print("Time: " + str(time.time() - start))
    print("Reading files...")
    time.sleep(2)
    for _f in tqdm(filelist):
            ds = pydicom.dcmread(_f, stop_before_pixels=True)
            items = []
            items.append(_f)

            for _tag in list_of_tags:
                if _tag in ds:
                    items.append(ds.data_element(_tag).value)
                else:
                    items.append("Not found")

            table.append((items))
    
    list_of_tags.insert(0, "Filename")
    test = list(map(list, zip(*table)))
    dictone = {}

    for i, _tag in enumerate (list_of_tags):
        dictone[_tag] = test[i]

    df = pd.DataFrame(dictone)
    time.sleep(2)
    print("Finished.")
    return df


# Basic File Visualition

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

!pip install kaggle

from google.colab import files
files.upload()

Mounted at /content/gdrive


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"escyubinhong","key":"0ebc417993db73eb003575ab871df771"}'}

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d ibombonato/xray-body-images-in-png-unifesp-competion

 99% 198M/200M [00:01<00:00, 129MB/s]
100% 200M/200M [00:01<00:00, 126MB/s]


In [5]:
%cd /content

!unzip -qq "/content/xray-body-images-in-png-unifesp-competion.zip"

/content


In [6]:
def convert_to_list(l):
        l=l.split()
        for n in range(len(l)):
            l[n]=int(l[n])
        return l

    
def display_(df_path,train_dir):
    tags = ['PhotometricInterpretation','BitsAllocated', 'SOPInstanceUID' ]
    dicom_tags_train =  dcmtag2table(train_dir, tags)
    df=pd.read_csv(df_path)
    df = dicom_tags_train.merge(df, on =  'SOPInstanceUID')
    df.Target=df.Target.map(lambda x: convert_to_list(x))
    display(df)
    print(df.info())
    return df


def plot_images(paths,r=8,c=8,figsize=(20,20)):
    
    _,axs=plt.subplots(r,c,figsize=figsize)
    axs=axs.flatten()
    
    for n, ax in enumerate(axs):
        img=read_img(paths[n])
        ax.imshow(img,cmap='gray')
        ax.axis('off')
        
    plt.tight_layout()
    plt.show()
    
def read_img(path):
    dcm=pydicom.dcmread(path)
    img=dcm.pixel_array
    return img
    
def plot_imgs(imgs,r=8,c=8,figsize=(20,20)):
    _,axs=plt.subplots(8,c,figsize=figsize)
    axs=axs.flatten()
    for n, ax in enumerate(axs):
        ax.imshow(torchvision.transforms.functional.to_pil_image(imgs[n]),cmap='gray')
        ax.axis('off')
        
    plt.tight_layout()
    plt.show()

In [8]:
def main():
    print('#'*100)
    print('training images')
    
    train_df_path='/content/train_df.csv'
    test_df_path='/content/test_df.csv'
    train_dir='/content/images/train'
    test_dir='/content/images/test'
    
    train_df=display_(train_df_path,train_dir)
    print(train_df.Filename[1])
    plot_images(train_df.Filename)
    
    print('#'*100)
    print('test images')
    
    test_df=display_(test_df_path,test_dir)
    plot_images(test_df.Filename)
    




main()

####################################################################################################
training images
Listing all files...
Time: 0.005067348480224609
Reading files...


  0%|          | 0/1738 [00:00<?, ?it/s]


InvalidDicomError: ignored

# Iput Pipeline

In [ ]:
class base_pipe(torch.utils.data.Dataset):
    
    def __init__(
        self,
        df
    ):
        super().__init__()
        self.df=df.reset_index()
        
    def __len__(self):
        return len(self.df)
    
    def get_label(self,label):
        label=torch.nn.functional.one_hot(torch.tensor(label),num_classes=22).sum(axis=0)
        label=torch.tensor(label,dtype=torch.float32)
        return label
    
    def read_img(self,path):
        t=torchvision.transforms.Resize(size=(512,512))
        dcm=pydicom.dcmread(path)
        img=dcm.pixel_array
        img=img.astype(np.float32)
        img=torch.tensor([img],dtype=torch.float32)
        img=img.repeat(3,1,1)
        img=t(img)
        return img/img.max()
    
    
    def __getitem__(self, idx):
        img=self.read_img(self.df.Filename[idx])
        label=self.get_label(self.df.Target[idx])
        return img,label

In [ ]:
class pipeline(pl.LightningDataModule):
    def __init__(
        self,
        ds,
        df,
        test_df,
        bs=5
    ):
        super().__init__()
        self.ds=ds
        self.train_df, self.val_df = train_test_split(df, test_size=0.2)
        self.test_df=test_df
        self.bs=bs
        
    def train_dataloader(self):
        ds=self.ds(self.train_df)
        dataloader=torch.utils.data.DataLoader(ds,batch_size=self.bs)
        return dataloader
    
    def val_dataloader(self):
        ds=self.ds(self.val_df)
        dataloader=torch.utils.data.DataLoader(ds,batch_size=self.bs)
        return dataloader
    
    def test_dataloader(self):
        ds=self.ds(self.test_df)
        dataloader=torch.utils.data.DataLoader(ds,batch_size=self.bs)
        return dataloader

# Model and Classifier

In [ ]:
   
class Base_Model(pl.LightningModule):
    
    def __init__(self,
                 model
                ):
        super().__init__()
        self.model=model
        self.model.classifier=torch.nn.Sequential(
            
            torch.nn.Linear(in_features=self.model.classifier.in_features,out_features=512),
            torch.nn.Dropout(0.3),
            torch.nn.ReLU(),
            torch.nn.Linear(512,128),
            torch.nn.Dropout(0.3),
            torch.nn.ReLU(),
            torch.nn.Linear(128,22),
            torch.nn.Softmax()
        )
        
        
    def forward(self,x):
        out=self.model(x)
        return out
    
class Classifier(pl.LightningModule):
    def __init__(
        self,
        model,
        loss,
        lr
    ):
        super().__init__()
        self.model=model
        self.loss=loss
        self.lr=lr

        
    def training_step(self,batch,batch_idx):
        imgs,targets=batch
        preds=self.model(imgs)
        loss=self.loss(preds,targets)
        self.log('train_loss',loss)
        return loss
        
    def valdation_step(self,batch,batch_idx):
        imgs,targets=batch
        preds=self.model(imgs)
        val_loss=self.loss(preds,targets)
        self.log('val_loss',val_loss)
        return val_loss
    
    def test_step(self,batch,batch_idx):
        imgs,targets=batch
        preds=self.model(imgs)
        test_loss=self.loss(preds,targets)
        self.log('test_loss',test_loss)
        return test_loss
        
    def configure_optimizers(self):
        opt=torch.optim.Adam(self.model.parameters(),lr=self.lr)
        return opt

# Let the fireworks begin!!

In [ ]:
def main():
    
    print('#'*100)
    print('training images')

    train_df_path='../input/unifesp-x-ray-body-part-classifier/train.csv'
    test_df_path='../input/unifesp-x-ray-body-part-classifier/sample_submission.csv'
    train_dir='../input/unifesp-x-ray-body-part-classifier/train/train'
    test_dir='../input/unifesp-x-ray-body-part-classifier/test/test'

    train_df=display_(train_df_path,train_dir)
    test_df=display_(test_df_path,test_dir)
    
    #WandB Logger
#     wandb_logger=pl.loggers.WandbLogger(name='Exp_1_1',project='X-Ray Body part',job_type='EXP_1')

    #Input Pipeline
    dataloader=pipeline(base_pipe,train_df,test_df,8)
    
    #Model
    Model_=torchvision.models.densenet121(pretrained=True,progress=False)
    model=Base_Model(Model_)
    
    #Classifer Logic
    loss=torch.nn.BCELoss()
    lr=1e-4
    classifier=Classifier(model,loss,lr)
    
    
    #Trainer
    trainer=pl.Trainer(max_epochs=1,
                       accelerator='gpu',
#                        logger=wandb_logger,
                       log_every_n_steps=1
                      )
    
    trainer.fit(classifier,dataloader)


main()

    
    

####################################################################################################
training images
Listing all files...
Time: 1.4691333770751953
Reading files...


100%|██████████| 1738/1738 [00:03<00:00, 523.60it/s]


Finished.


,Filename,PhotometricInterpretation,BitsAllocated,SOPInstanceUID,Target
0,../input/unifesp-x-ray-body-part-classifier/tr...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.71157989004260882669...,[3]
1,../input/unifesp-x-ray-body-part-classifier/tr...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.32467620439025796224...,[4]
2,../input/unifesp-x-ray-body-part-classifier/tr...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.74856220852423198555...,[4]
3,../input/unifesp-x-ray-body-part-classifier/tr...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.90865692473901867788...,[7]
4,../input/unifesp-x-ray-body-part-classifier/tr...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.44687741644515558201...,[21]
...,...,...,...,...,...
1733,../input/unifesp-x-ray-body-part-classifier/tr...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.84195286770915865285...,[10]
1734,../input/unifesp-x-ray-body-part-classifier/tr...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.11211536623788107002...,[10]
1735,../input/unifesp-x-ray-body-part-classifier/tr...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.97523800054763336883...,[21]
1736,../input/unifesp-x-ray-body-part-classifier/tr...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.13690494879381638155...,[21]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1738 entries, 0 to 1737
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Filename                   1738 non-null   object
 1   PhotometricInterpretation  1738 non-null   object
 2   BitsAllocated              1738 non-null   int64 
 3   SOPInstanceUID             1738 non-null   object
 4   Target                     1738 non-null   object
dtypes: int64(1), object(4)
memory usage: 81.5+ KB
None
Listing all files...
Time: 0.6478805541992188
Reading files...


...

Finished.


,Filename,PhotometricInterpretation,BitsAllocated,SOPInstanceUID,Target
0,../input/unifesp-x-ray-body-part-classifier/te...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.81525197438350521711...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,../input/unifesp-x-ray-body-part-classifier/te...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.69289037696158311261...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
2,../input/unifesp-x-ray-body-part-classifier/te...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.32891312658070649342...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
3,../input/unifesp-x-ray-body-part-classifier/te...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.23544226701264511758...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
4,../input/unifesp-x-ray-body-part-classifier/te...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.10388310349619026152...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
...,...,...,...,...,...
738,../input/unifesp-x-ray-body-part-classifier/te...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.64695143207725149148...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
739,../input/unifesp-x-ray-body-part-classifier/te...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.17725594983911803945...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
740,../input/unifesp-x-ray-body-part-classifier/te...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.61422426455206487334...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
741,../input/unifesp-x-ray-body-part-classifier/te...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.37411450617057499635...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."


<class 'pandas.core.frame.DataFrame'>
Int64Index: 743 entries, 0 to 742
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Filename                   743 non-null    object
 1   PhotometricInterpretation  743 non-null    object
 2   BitsAllocated              743 non-null    int64 
 3   SOPInstanceUID             743 non-null    object
 4   Target                     743 non-null    object
dtypes: int64(1), object(4)
memory usage: 34.8+ KB
None


Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


Training: 0it [00:00, ?it/s]